In [ ]:
#default=[1,2,3,4,5,6]
default=[2]

In [ ]:
import argparse
import configparser
import sys

from datetime import datetime

import torch

import conf
from db import InfluxDB, datetime_to_nanosec
from network import Network
from swat_dataset import SWaTDataset


#assert torch.cuda.device_count() >= 1

BATCH_SIZE = 4096

org = "Osaka"

DB = InfluxDB('swat',org)

#import influxdb_client
#from influxdb_client.client.write_api import SYNCHRONOUS

#bucket = "swat"
#org = "Osaka"
#token = "2g_Q50vP5zhuxKx-y8Z3YoMBlRJA7t-c1C9c446mtrF4uUULh1ikdYy7MIBgsiY3NsAbxM5JasJKkm3HbdINsA=="
# Store the URL of your InfluxDB instance
#url="http://localhost:8086"

#client = influxdb_client.InfluxDBClient(
#    url=url,
#    token=token,
#    org=org
#)

#write_api = client.write_api(write_options=SYNCHRONOUS)


#parser = argparse.ArgumentParser(description='Trainer')

#parser.add_argument('--gpu', type=int, help='GPU index for learning (1-6)')
#parser.add_argument('--save', type=int, help='The index of trained network')
#args = parser.parse_args()

#print(f'* target processes: {args.process}')
#print(f'* use CUDA with GPU #{args.gpu}')
#print(f'* N-programming index: #{args.save}')

config = configparser.ConfigParser()
config.read('config.ini')
training_conf = config['train']

for pidx in default:
    print(f'* starts to train process {pidx}')

    dataset = SWaTDataset('dat/normal-P{}.dat'.format(pidx))

    p_features = len(conf.P_SRCS[pidx - 1])
    print(f'* # of features: {p_features}')
    pnet = Network(pidx=pidx, gidx=1, n_features=p_features, n_hiddens=conf.N_HIDDEN_CELLS)

    influx_measurement = conf.TRAIN_LOSS_MEASUREMENT.format(pidx)
    print(influx_measurement)
    DB.clear(influx_measurement)

    training_start = datetime.now()
    epochs = int(training_conf['epochs'])
    print(f'* training is going to repeat {epochs:,} times (epochs)')
    pnet.train_mode()

    min_loss = sys.float_info.max
    train_losses = []
    for e in range(epochs):
        
        start = datetime.now()
        loss = pnet.train(dataset, BATCH_SIZE)
        print(f'printing loss {loss}')
        #train_losses.append(loss)
        #globals()[f'p{e}'] = influxdb_client.Point(influx_measurement).tag(" ", " ").field('train_loss_{}', loss)
        #write_api.write(bucket=bucket, org=org, record= globals()[f'p{e}'])
        
        #DB.write(
        #    influx_measurement,
        #    {},
        #    {'train_loss_{}'.format(1): [loss]},
        #    [datetime_to_nanosec(datetime.now())]
        #)
        print(f'train loss {train_losses}')
        saved = False
        if loss < min_loss:
            min_loss = loss
            pnet.save(1, min_loss)
            saved = True
        print(f'[{e+1:>4}] {loss:10.6} ({datetime.now() - start})' + (' -> saved' if saved else ''))
    print(f'* the total training time: {datetime.now() - training_start}')

In [ ]:
for i in range(0,len(train_losses)):
    print(train_losses[i])


In [ ]:
for i in range(10):
    globals()[f'p{i}']=i
    
print(p2)

In [ ]:
import matplotlib.pyplot as plt
ax = plt.figure().gca()

ax.plot(train_losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend('train')
plt.title('Loss over training epochs')
plt.show();

In [ ]:
import argparse
import sys
from datetime import datetime

import torch
from torch.utils.data import ConcatDataset

import conf
from db import InfluxDB, datetime_to_nanosec
from network import Network
from swat_dataset import SWaTDataset

N_DUPLICATE_RUNS = 2

BATCH_SIZE = 1024  # larger validation batch is possible, but DB is the bottleneck
#DB = InfluxDB('swat')

#parser = argparse.ArgumentParser(description='validator')
#parser.add_argument('--process', type=int, help='Process index (1-6)')
#args = parser.parse_args()

#pidx = args.process
pidx = 1
assert 1 <= pidx <= 6
#DB.clear(conf.EVAL_MEASUREMENT.format(pidx))

p_features = len(conf.P_SRCS[pidx - 1])
pnet = Network(pidx=pidx, gidx=pidx, n_features=p_features, n_hiddens=conf.N_HIDDEN_CELLS)

current_min_loss = sys.float_info.max
min_idx = 1
#for i in range(N_DUPLICATE_RUNS):
    #min_loss = pnet.load(i + 1)
    #print(f'* loaded saved network #{i+1}, min_loss: {min_loss}')
    #if min_loss < current_min_loss:
        #min_idx = i + 1
        #current_min_loss = min_loss
print(f'* the minimum loss index: {min_idx}')
pnet.load(min_idx)

dataset = ConcatDataset(
    [SWaTDataset('dat/normal-P{}.dat'.format(pidx)),
     SWaTDataset('dat/attack-P{}.dat'.format(pidx))]
)

pnet.eval_mode()
start = datetime.now()
val_losses = []
with torch.no_grad():
    loss = pnet.eval(dataset, BATCH_SIZE, db_write=False)
#DB.write(conf.TRAIN_LOSS_MEASUREMENT.format(pidx), {}, {'val_loss': [loss]}, [datetime_to_nanosec(datetime.now())])
val_losses.append(loss)
print(f'* val loss: {loss} ({datetime.now() - start})')

In [ ]:
for i in range(0,len(val_losses)):
    print(val_losses[i])

In [ ]:
import argparse
import configparser
from collections import deque
from datetime import datetime, timedelta

import numpy as np

import conf
from db import InfluxDB, datetime_to_nanosec

DB = InfluxDB('swat')
FETCH_SIZE = 1_000


class Smooth:
    def __init__(self, start: datetime, pidx: int, n_aggregate: int, n_cutoff: int):
        self.db_cursor = start
        self.pidx = pidx
        assert n_aggregate > n_cutoff * 2
        self.n_aggregate = n_aggregate
        self.n_cutoff = n_cutoff
        self.bucket: deque = deque(maxlen=n_aggregate)
        self.moving_avg: deque = deque(maxlen=n_aggregate)
        self.buffer: deque = deque(maxlen=FETCH_SIZE)
        while len(self.bucket) < n_aggregate:
            self.push()

    def fetch_to_buffer(self) -> int:
        r = DB.read(conf.EVAL_MEASUREMENT.format(1), self.db_cursor, seconds=FETCH_SIZE).json()
        print(r)
        count = 0
        last_time_str = ''
        for statement in r['results']:
            for s in statement['series']:
                cols = s['columns']
                for v in s['values']:
                    v_dict = dict(zip(cols, v))
                    self.buffer.append(v_dict)
                    last_time_str = v_dict['time']
                    count += 1
        if last_time_str != '':
            self.db_cursor = datetime.strptime(last_time_str,
                                               conf.INFLUX_RETURN_TIME_FORMAT) + timedelta(hours=9
                                                                                           ) + timedelta(seconds=1)
        print(f'\t- fetched {count:,} datapoints')
        return count

    def push(self):
        while True:
            try:
                self.bucket.append(self.buffer.popleft())
                break
            except IndexError:  # when buffer is empty
                self.fetch_to_buffer()

    def check(self, high: int, low: int, ratio: int):
        dist_window = np.array([v['distance'] for v in self.bucket], dtype=np.float32)
        dist_window = np.delete(dist_window, dist_window.argsort()[-self.n_cutoff:])
        self.moving_avg.append(np.sum(dist_window))
        sus = 0
        # write = True
        if len(self.moving_avg) == self.n_aggregate:
            avg_win = np.array(self.moving_avg, dtype=np.float32)
            sus = min(np.percentile(avg_win, high) / np.percentile(avg_win, low) / ratio, 1.0)
            # i_high = abs(avg_win - np.percentile(avg_win, high, interpolation='lower')).argmin()
            # i_low = abs(avg_win - np.percentile(avg_win, low, interpolation='lower')).argmin()
            # write = i_high > i_low
        return datetime.strptime(self.bucket[-1]['time'], '%Y-%m-%dT%H:%M:%SZ') + timedelta(hours=9), self.bucket[-1]['attack'], sus


In [1]:
from swat_dataset import SWaTDataset

In [ ]:
dataset1 = SWaTDataset('dat/normal-P1.dat')

In [ ]:
print(dataset.__getitem__)

In [ ]:
first = dataset1[0]
print(first)

In [ ]:
for x in first.keys():
    print(x)

In [ ]:
firstgiven = first['given']
print(firstgiven)

In [ ]:
firstgiven[0,1:5]

In [ ]:
print(dataset[2])

In [2]:
dataset = SWaTDataset('dat/normal-P5.dat')


In [ ]:
first1 =dataset[0]
for x in first1:
    print(x)

In [ ]:
first1

In [ ]:
import torch 
from torch.utils.data import DataLoader
counter = 0
for i in DataLoader(dataset,batch_size=4096,shuffle=False):
    
    given = i['given']
    predict = i['predict']
    answer = i['answer']
    counter = counter + 1
    print(given.shape)#x
    print(predict.shape)
    print(answer.shape)
    #dxp = torch.zeros(given.shape)
    #print(dxp)
    
    
    print(counter)
    
    #print(predict.shape)#xval
    #print(answer.shape) # y
    



In [ ]:
dxp.shape

In [ ]:
dxp

In [ ]:
batch_size = given.size(0)
batch_size

In [ ]:
given = given.view(batch_size, 90, -1)
given = given.transpose(0,1)
given.shape

In [ ]:
first_data = dataset[0]

In [ ]:
ts, given, predict, answer, attack = first_data

In [ ]:
first_data[:,4:8]

In [ ]:
first_data['given']

In [ ]:
first_data['attack'].shape

In [ ]:
first_data['given'].shape

In [ ]:
first_data['predict'].shape

In [ ]:
first_data['answer'].shape

In [ ]:
first_data['given']

In [ ]:
data_sample = first_data['given']

In [ ]:
data_sample.shape

In [ ]:
data_sample[:,4:8]

In [ ]:
def attack_feature(sensorID,num):
    l=list()
    for i in range(0,10*num,10):
        if sensorID==0:
            for j in range(0,5):
                l.append(i+j)
        if sensorID==1:
            for j in range(5,10):
                l.append(i+j)
        
    return l

In [ ]:
li_attack = attack_feature(int(0),4)
li_attack

In [ ]:
len(li_attack)

In [ ]:
import numpy as np
def MaskForAttackPoints(size):
        a=np.zeros(size[1])
        for i in range(0,13):
            a[i]=1
        return np.tile(a,(size[0],1)).astype(np.float32)

In [ ]:
mask = MaskForAttackPoints(dataset[0]['given'].shape)

In [ ]:
mask

In [ ]:
mask.shape

In [8]:
import copy 
def ObtainListOfParameters(Model):
    l=list()
    for w in Model.parameters():
        l.append(copy.deepcopy(w))
    return l

In [ ]:
def UpdateModel(Model, Alpha,MaxGrad=1,ListOfParam=None):
    if ListOfParam is None:
        for w in Model.parameters():
            w.data=w.data + Alpha* w.grad
    else:
        i=0
        for w in Model.parameters():
            w.data=w.data +Alpha * ListOfParam[i].grad
            i=i+1

In [10]:
def GetLossFunc(batch, enc, dec):
    given = batch['given'].to(torch.device('cuda:0'))
    predict = batch['predict'].to(torch.device('cuda:0'))
    answer = batch['answer'].to(torch.device('cuda:0')) 
    encoder_outs, context = enc(given)
    guess = dec(encoder_outs, context, predict)
    mse_loss = nn.MSELoss()
    loss = mse_loss(guess, answer)

    return loss 

In [29]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

import conf
import model
import numpy as np 
from db import InfluxDB, swat_time_to_nanosec

DB = InfluxDB('swat')


class Network():
    def __init__(self, pidx: int, gidx: int, n_features: int, n_hiddens: int):
        self.n_features = n_features
        self.pidx = pidx
        self.gidx = gidx
        self.gpu = torch.device('cuda:{}'.format(gidx - 1))

        self.encoder = model.Encoder(n_inputs=n_features, n_hiddens=n_hiddens).to(self.gpu)
        self.decoder = model.AttentionDecoder(n_hiddens=n_hiddens, n_features=n_features).to(self.gpu)

        self.model_fn = 'checkpoints/SWaT-P{}'.format(pidx)

        self.encoder_optimizer = optim.Adam(self.encoder.parameters(), amsgrad=True)
        self.decoder_optimizer = optim.Adam(self.decoder.parameters(), amsgrad=True)

        self.mse_loss = nn.MSELoss()
    #--------------------------------------------------------------------------------------------------------------
    def get_loss_func(self,x,y):

        loss = self.mse_loss(x, y)

        return loss

    #--------------------------------------------------------------------------------------------------------------
    def GetGradientUsingAutoGrad(enc, dec, TrainingData,batch_size):
        givenbag = []
        predictbag = []
        for batch in DataLoader(TrainingData, batch_size=batch_size, shuffle=True):
            (loss, given, predict) = GetLossFunc(batch, enc, dec)
            given1 = given.clone().detach().requires_grad_(True)
            predict1 = predict.clone().detach().requires_grad_(True)
            grad = torch.autograd.grad(outputs=loss, inputs=(given,predict), retain_graph=True, allow_unused=True)
            loss.backward()
            givenbag.append(given.grad)
            predictbag.append(predict.grad)
        return (grad,enc, dec, givenbag, predictbag)
    
    #--------------------------------------------------------------------------------------------------------------
    def train(self, dataset, batch_size) -> float:
        epoch_loss = 0
        counter = 0
        bag = []
        bag2 = []
        basket = []
        for batch in DataLoader(dataset, batch_size=batch_size, shuffle=True):
            self.encoder_optimizer.zero_grad()
            self.decoder_optimizer.zero_grad()
            given = batch['given'].to(self.gpu)
            predict = batch['predict'].to(self.gpu)
            answer = batch['answer'].to(self.gpu) 
            given = given.clone().detach().requires_grad_(True)
            predict = predict.clone().detach().requires_grad_(True)
            encoder_outs, context = self.encoder(given)
            guess = self.decoder(encoder_outs, context, predict)
            mse_loss = nn.MSELoss()
            loss = mse_loss(guess, answer)
            
            #---------------------------------------------------------
            
          # grad = torch.autograd.grad(outputs=loss, inputs=given,retain_graph=True, allow_unused=True)
          # grad = torch.autograd.grad(outputs=loss, inputs=predict,retain_graph=True, allow_unused=True)      
            grad = torch.autograd.grad(outputs=loss, inputs=(given,predict),retain_graph=True, allow_unused=True)
          # print(grad)
            print('------------------')
            print(given.grad)
          # w = ObtainListOfParameters(self.encoder)
          # print(w)
            
          # print(given)
          # print(predict)
          # print(given.grad)
            counter = counter + 1
            print(counter)
           #loss.backward(inputs= [given,predict])
            loss.backward()
            print('---------------------after backward------------')
            print(given.grad)
           
            #poch_loss += loss.item()
            #epoch_loss += loss.item()
            #answer, guess = self.infer(batch)
            #loss = self.mse_loss(guess, answer)
            #loss = self.get_loss_func(guess, answer)
            #grad = torch.autograd.grad(outputs=loss, inputs=given)
            #loss.backward() # just calculates the gradient 
            #print("loss.data")
            #print(loss.data)
            epoch_loss += loss.item()
            self.encoder_optimizer.step()
            self.decoder_optimizer.step()
          # print(given.grad)
            print('------------------------------------------------------')
        #   a = predict.grad
        #   b = predict.grad
        #   c = a - b
        #   print(predict.grad)
        #   print('---------------------shape-----------------------')
        #   print(given.grad.shape)
        #   print('---------------------subtract---------------')
        #   print(c)
        #   print(c.shape)
        #   dxp = torch.zeros(given.shape)
         #  print(dxp)
         #  print(dxp.shape)
            
         #  bag.append(predict.grad)
            
          # bag = torch.as_tensor(bag)
        
        print('12311111111111111111111111111111111111111111111111111')
       #print(bag)
       #for i in range(len(bag)):
           #basket[i] = bag[i] - bag[i]
           #print(basket[i])
          # print(bag[i]-bag[i])
         #  basket.append(bag[i]-bag[i])
           #print(basket[i])
           #print('---------------------shape-----------------------')
            #rint(basket[i].shape)
            
        #rint(len(basket))
        return epoch_loss

    def eval(self, dataset, batch_size, db_write: bool) -> float:
        epoch_loss = 0
        n_datapoints = 0
        for batch in DataLoader(dataset, batch_size=batch_size, shuffle=False):
            ts = [swat_time_to_nanosec(t) for t in batch['ts']]
            attack = batch['attack']

            answer, guess = self.infer(batch)
            distance = torch.norm(guess - answer, p=conf.EVALUATION_NORM_P, dim=1)
            epoch_loss += torch.sum(distance).item()
            n_datapoints += len(ts)

            if db_write:  # write all distances to influxDB
                col_dist = torch.abs(guess - answer).cpu().numpy()
                fields = {k: col_dist[:, i] for i, k in enumerate(conf.P_SRCS[self.pidx - 1])}
                fields.update({'distance': distance.cpu().tolist()})
                DB.write(conf.EVAL_MEASUREMENT.format(self.pidx), {'attack': attack}, fields, ts)
        return epoch_loss / n_datapoints

    def infer(self, batch):
        given = batch['given'].to(self.gpu)
        predict = batch['predict'].to(self.gpu)
        answer = batch['answer'].to(self.gpu)
        encoder_outs, context = self.encoder(given)
        guess = self.decoder(encoder_outs, context, predict)
        return answer, guess

    def load(self, idx: int) -> float:
        fn = self.model_fn + '-{}.net'.format(idx)
        checkpoint = torch.load(fn)
        self.encoder.load_state_dict(checkpoint['model_encoder'])
        self.decoder.load_state_dict(checkpoint['model_decoder'])
        self.encoder_optimizer.load_state_dict(checkpoint['optimizer_encoder'])
        self.decoder_optimizer.load_state_dict(checkpoint['optimizer_decoder'])
        return checkpoint['min_loss']

    def save(self, idx: int, min_loss: float) -> None:
        fn = self.model_fn + '-{}.net'.format(idx)
        torch.save(
            {
                'min_loss': min_loss,
                'model_encoder': self.encoder.state_dict(),
                'model_decoder': self.decoder.state_dict(),
                'optimizer_encoder': self.encoder_optimizer.state_dict(),
                'optimizer_decoder': self.decoder_optimizer.state_dict(),
            },
            fn
        )

    def train_mode(self) -> None:
        self.encoder.train()
        self.decoder.train()

    def eval_mode(self) -> None:
        self.encoder.eval()
        self.decoder.eval()

In [30]:
pnet = Network(pidx=5, gidx=1, n_features=13, n_hiddens=64)

In [31]:
pnet.train(dataset,batch_size=4096)

------------------
None
1
---------------------after backward------------
tensor([[[ 6.4968e-11, -4.5542e-11, -2.8061e-11,  ...,  2.6130e-11,
           1.6129e-11, -1.1600e-10],
         [ 6.6327e-11, -5.1376e-11, -2.5034e-11,  ...,  3.3448e-11,
           1.6713e-11, -1.2254e-10],
         [ 6.5777e-11, -5.3614e-11, -2.3005e-11,  ...,  3.6406e-11,
           1.6467e-11, -1.2377e-10],
         ...,
         [-3.1613e-13, -5.5044e-11, -4.6572e-11,  ...,  1.5555e-11,
           6.9936e-11, -1.9843e-11],
         [ 3.6658e-13, -3.8295e-11, -3.8390e-11,  ...,  7.3602e-12,
           4.9376e-11, -1.2091e-11],
         [ 1.7694e-12, -1.6958e-11, -2.0280e-11,  ...,  2.4649e-12,
           2.2091e-11, -5.0203e-12]],

        [[ 4.3249e-11, -1.8272e-11, -5.7661e-11,  ..., -1.9255e-11,
           1.1635e-12, -8.2033e-11],
         [ 4.5947e-11, -2.2059e-11, -5.7656e-11,  ..., -1.3936e-11,
           1.3890e-12, -8.6675e-11],
         [ 4.6413e-11, -2.3427e-11, -5.6842e-11,  ..., -1.1275e-11,
  

------------------
None
4
---------------------after backward------------
tensor([[[-1.3098e-11, -7.4146e-11, -1.1966e-10,  ...,  2.3158e-12,
           3.1040e-11, -6.5521e-11],
         [-1.4173e-11, -8.3009e-11, -1.1254e-10,  ...,  1.3343e-11,
           3.7597e-11, -7.1300e-11],
         [-1.4827e-11, -8.6929e-11, -1.0769e-10,  ...,  1.8024e-11,
           4.1553e-11, -7.4137e-11],
         ...,
         [-5.6185e-11, -7.4705e-11, -8.8346e-11,  ...,  9.1814e-12,
           9.0352e-11,  1.3368e-12],
         [-3.8629e-11, -4.9866e-11, -6.6327e-11,  ...,  1.2065e-12,
           6.0786e-11,  1.2828e-12],
         [-1.6349e-11, -2.0883e-11, -3.2469e-11,  ..., -1.5884e-12,
           2.5757e-11,  9.5931e-13]],

        [[-1.0059e-11, -8.2096e-11, -1.3399e-10,  ...,  1.1226e-12,
           2.7141e-11, -7.9412e-11],
         [-1.1117e-11, -9.1399e-11, -1.2597e-10,  ...,  1.3122e-11,
           3.4292e-11, -8.6082e-11],
         [-1.1870e-11, -9.5543e-11, -1.2061e-10,  ...,  1.8155e-11,
  

------------------
None
7
---------------------after backward------------
tensor([[[-1.8585e-10, -2.2245e-10, -4.1041e-10,  ..., -1.1847e-10,
           3.9643e-12, -2.8319e-10],
         [-1.9649e-10, -2.5254e-10, -3.7997e-10,  ..., -8.1548e-11,
           2.8912e-11, -2.9973e-10],
         [-2.0627e-10, -2.6740e-10, -3.6119e-10,  ..., -6.5194e-11,
           4.2360e-11, -3.0896e-10],
         ...,
         [-1.2087e-10, -1.3509e-10, -1.9522e-10,  ...,  3.2146e-12,
           1.3922e-10, -1.1858e-11],
         [-7.4875e-11, -8.2094e-11, -1.3933e-10,  ..., -6.6144e-12,
           9.1715e-11,  1.9991e-12],
         [-3.0060e-11, -3.0841e-11, -6.5595e-11,  ..., -7.6091e-12,
           3.7885e-11,  5.0839e-12]],

        [[-1.8787e-10, -2.2723e-10, -4.1680e-10,  ..., -1.1681e-10,
           3.0611e-12, -2.8898e-10],
         [-1.9899e-10, -2.5699e-10, -3.8681e-10,  ..., -8.0162e-11,
           2.7950e-11, -3.0558e-10],
         [-2.0922e-10, -2.7204e-10, -3.6807e-10,  ..., -6.3580e-11,
  

------------------
None
10
---------------------after backward------------
tensor([[[-1.2420e-09, -9.5178e-10, -1.2859e-09,  ..., -6.3489e-10,
          -3.9415e-10, -1.4110e-09],
         [-1.3464e-09, -1.0765e-09, -1.1852e-09,  ..., -5.1011e-10,
          -3.4016e-10, -1.4630e-09],
         [-1.4358e-09, -1.1442e-09, -1.1199e-09,  ..., -4.5156e-10,
          -3.2143e-10, -1.4959e-09],
         ...,
         [-2.2866e-10, -2.1273e-10, -3.1797e-10,  ..., -1.2851e-11,
           1.6788e-10, -5.7867e-11],
         [-1.0037e-10, -1.0240e-10, -2.2319e-10,  ..., -2.5886e-11,
           1.2202e-10,  3.9886e-12],
         [-2.4146e-11, -2.6501e-11, -1.0481e-10,  ..., -2.2166e-11,
           5.5022e-11,  1.8400e-11]],

        [[-1.3515e-09, -1.0383e-09, -1.2425e-09,  ..., -7.1554e-10,
          -4.2090e-10, -1.6089e-09],
         [-1.4685e-09, -1.1890e-09, -1.1101e-09,  ..., -5.6527e-10,
          -3.5941e-10, -1.6734e-09],
         [-1.5705e-09, -1.2677e-09, -1.0286e-09,  ..., -4.9694e-10,
 

------------------
None
13
---------------------after backward------------
tensor([[[-4.0088e-09, -2.6303e-09, -2.5494e-09,  ..., -2.0738e-09,
          -1.5342e-09, -4.0877e-09],
         [-4.4150e-09, -3.0441e-09, -2.2842e-09,  ..., -1.7561e-09,
          -1.4858e-09, -4.2209e-09],
         [-4.7371e-09, -3.2445e-09, -2.1074e-09,  ..., -1.6010e-09,
          -1.5058e-09, -4.2697e-09],
         ...,
         [ 6.2053e-12, -7.1635e-11, -1.8202e-10,  ...,  3.1171e-11,
           1.9574e-10,  6.5903e-11],
         [ 1.1269e-10,  6.8858e-12, -1.4580e-10,  ...,  5.0423e-12,
           1.7346e-10,  1.0950e-10],
         [ 8.8288e-11,  2.2423e-11, -8.5160e-11,  ..., -8.8513e-12,
           9.4755e-11,  7.1784e-11]],

        [[-4.1015e-09, -2.7110e-09, -2.6651e-09,  ..., -1.9752e-09,
          -1.4423e-09, -3.9764e-09],
         [-4.4942e-09, -3.1015e-09, -2.4030e-09,  ..., -1.6490e-09,
          -1.3458e-09, -4.0933e-09],
         [-4.8008e-09, -3.2903e-09, -2.2313e-09,  ..., -1.4855e-09,
 

...

------------------
None
112
---------------------after backward------------
tensor([[[-6.2762e-12, -4.2287e-12, -3.5105e-12,  ..., -3.0905e-12,
          -3.2800e-12, -4.4895e-12],
         [-2.5326e-12, -1.6954e-12, -1.1912e-12,  ..., -1.0814e-12,
          -1.2749e-12, -1.6047e-12],
         [-7.1479e-13, -4.1778e-13, -2.4361e-13,  ..., -3.0656e-13,
          -3.8520e-13, -3.5655e-13],
         ...,
         [ 2.3995e-15,  1.4892e-14, -1.3598e-13,  ...,  7.0560e-14,
           1.4403e-14,  1.3903e-13],
         [-1.8553e-14, -1.7879e-15, -1.7723e-13,  ...,  1.0740e-13,
          -8.1990e-15,  1.2276e-13],
         [-4.5235e-14,  2.6701e-14, -1.8660e-13,  ...,  1.4261e-13,
          -4.0485e-14,  8.1738e-14]],

        [[ 8.5530e-12,  5.8280e-12,  4.9260e-12,  ...,  4.3078e-12,
           4.6921e-12,  6.5198e-12],
         [ 4.1822e-12,  2.8739e-12,  2.1399e-12,  ...,  1.8176e-12,
           2.2335e-12,  2.8783e-12],
         [ 1.6633e-12,  1.0653e-12,  7.6492e-13,  ...,  7.1291e-13,


------------------
None
115
---------------------after backward------------
tensor([[[ 1.0649e-11,  7.1671e-12,  6.1526e-12,  ...,  5.4041e-12,
           5.7601e-12,  7.9618e-12],
         [ 4.5636e-12,  3.1016e-12,  2.2928e-12,  ...,  2.0173e-12,
           2.3926e-12,  3.0669e-12],
         [ 1.4566e-12,  9.1031e-13,  6.0192e-13,  ...,  6.4616e-13,
           8.0147e-13,  8.1285e-13],
         ...,
         [ 1.1594e-12,  2.3027e-13,  5.5680e-13,  ...,  4.1032e-13,
           8.5883e-13, -2.4324e-14],
         [ 9.9026e-13,  1.3591e-13,  4.6963e-13,  ...,  3.4705e-13,
           7.4931e-13, -3.4140e-14],
         [ 6.0867e-13,  1.9476e-14,  3.1227e-13,  ...,  1.7900e-13,
           4.3185e-13, -2.8779e-14]],

        [[-1.7027e-11, -1.1631e-11, -9.4101e-12,  ..., -8.3071e-12,
          -8.8679e-12, -1.2128e-11],
         [-6.8438e-12, -4.6140e-12, -3.1474e-12,  ..., -2.8957e-12,
          -3.4340e-12, -4.3350e-12],
         [-1.9260e-12, -1.1288e-12, -6.1539e-13,  ..., -8.0431e-13,


------------------
None
118
---------------------after backward------------
tensor([[[ 8.3353e-12,  5.6452e-12,  4.7286e-12,  ...,  4.1750e-12,
           4.4915e-12,  6.2061e-12],
         [ 3.7352e-12,  2.5403e-12,  1.8306e-12,  ...,  1.6114e-12,
           1.9509e-12,  2.4854e-12],
         [ 1.2927e-12,  8.0249e-13,  5.3007e-13,  ...,  5.5024e-13,
           7.1323e-13,  7.1341e-13],
         ...,
         [ 9.7454e-13,  1.7450e-13,  3.9351e-13,  ...,  3.5889e-13,
           7.5151e-13,  4.6322e-14],
         [ 8.4217e-13,  1.0001e-13,  3.1461e-13,  ...,  3.2070e-13,
           6.5366e-13,  3.9924e-14],
         [ 5.2479e-13,  1.5691e-14,  1.8548e-13,  ...,  1.9902e-13,
           3.7567e-13,  2.4643e-14]],

        [[-9.4010e-12, -6.3568e-12, -5.2690e-12,  ..., -4.5813e-12,
          -4.8031e-12, -6.7151e-12],
         [-3.8229e-12, -2.5751e-12, -1.8107e-12,  ..., -1.6091e-12,
          -1.8739e-12, -2.4254e-12],
         [-1.1018e-12, -6.5220e-13, -3.8377e-13,  ..., -4.6078e-13,


------------------
None
121
---------------------after backward------------
tensor([[[ 1.1461e-11,  7.7518e-12,  6.5515e-12,  ...,  5.7666e-12,
           6.2247e-12,  8.5658e-12],
         [ 5.1359e-12,  3.4975e-12,  2.5495e-12,  ...,  2.2307e-12,
           2.7117e-12,  3.4415e-12],
         [ 1.7760e-12,  1.1114e-12,  7.4117e-13,  ...,  7.6071e-13,
           9.8715e-13,  9.9139e-13],
         ...,
         [ 1.6060e-12,  2.9661e-13,  6.0435e-13,  ...,  5.9253e-13,
           1.2297e-12,  1.1833e-13],
         [ 1.3719e-12,  1.7017e-13,  4.6078e-13,  ...,  5.3657e-13,
           1.0665e-12,  1.0207e-13],
         [ 8.3593e-13,  4.4028e-14,  2.5383e-13,  ...,  3.4330e-13,
           6.0749e-13,  6.5551e-14]],

        [[ 6.9653e-12,  4.7393e-12,  4.0051e-12,  ...,  3.4824e-12,
           3.7997e-12,  5.2611e-12],
         [ 3.3872e-12,  2.3192e-12,  1.7245e-12,  ...,  1.4565e-12,
           1.8008e-12,  2.3007e-12],
         [ 1.3327e-12,  8.4581e-13,  6.0564e-13,  ...,  5.6281e-13,


7.0267493291758

In [ ]:
(gradgiven, gradpredict, epoch_loss) = pnet.ObtainGradientofDataNew(TrainingData=dataset, epochs=3, batch_size=4096)

In [ ]:
epochs = 1
for e in range(epochs):
    #print(epochs)
    loss = pnet.train(dataset=dataset, batch_size=4096)
    print(loss)

In [92]:
def CreateZeroTensor(TrainingData, batch_size):
    givenzeros = []
    predictzeros = []
    onebasket = []
    for batch in DataLoader(TrainingData, batch_size=batch_size, shuffle=True):
        given = batch['given'].to(torch.device('cuda:0'))
        predict = batch['predict'].to(torch.device('cuda:0'))
        dgiven = torch.zeros(given.shape).to(torch.device('cuda:0'))
        onestensor = torch.ones(given.shape)
        dpredict = torch.zeros(predict.shape)
        givenzeros.append(dgiven)
        predictzeros.append(dpredict)
        onebasket.append(onestensor)
    
    return (givenzeros, predictzeros, onebasket)

In [93]:
(givenzeros, predictzeros, onebasket) = CreateZeroTensor(TrainingData=dataset, batch_size=4096)


In [109]:
b = givenzeros
print(len(b))

122


In [110]:
b[0] = givenzeros[0] +1.2
b[1] = givenzeros[1] + 1.3
b


[tensor([[[1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          [1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          [1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          ...,
          [1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          [1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          [1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000]],
 
         [[1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          [1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          [1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          ...,
          [1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          [1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          [1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000]],
 
         [[1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          [1.2000, 1.2000, 1.2000,  ..., 1.2000, 1.2000, 1.2000],
          [1.2000, 1.2000, 1.2000,  ...,

In [96]:
a = givenzeros

In [97]:
a = 

[tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 

In [5]:
encoder = model.Encoder(n_inputs=13, n_hiddens=64).to(torch.device('cuda:0'))
decoder = model.AttentionDecoder(n_hiddens=64, n_features=13).to(torch.device('cuda:0'))

In [14]:
def ObtainGradientOfWeightInModelNew(ModelEnc, ModelDec, TrainingData, batch_size, MaxGrad=100.0):
        enc = copy.deepcopy(ModelEnc)
        dec = copy.deepcopy(ModelDec)
        enc.zero_grad()
        dec.zero_grad()
        epoch_loss = 0

        for batch in DataLoader(TrainingData, batch_size=batch_size, shuffle=True):
            (loss, given, predict) = GetLossFunc(batch, enc, dec)
            loss.backward()
            epoch_loss += loss.item()
    
        for w in enc.parameters():
          #  print('----------------------wgrad----------------------')
          #  print(w.grad)
            if torch.max(torch.abs(w.grad))>MaxGrad:
                w.grad = w.grad/torch.max(torch.abs(w.grad))*MaxGrad
               # print('------------------wgradupdate-------------------')
               # print(w.grad)
    
        for w in dec.parameters():
            if torch.max(torch.abs(w.grad))>MaxGrad:
                w.grad = w.grad/torch.max(torch.abs(w.grad))*MaxGrad
    
        return (epoch_loss, enc, dec)

In [15]:
(epoch_loss, enc, dec) = ObtainGradientOfWeightInModelNew(encoder, decoder, TrainingData=dataset,batch_size=4096 )

In [64]:
def GetLossFunc(batch, enc, dec, autograd=True):
    if autograd:
        given = batch['given'].to(torch.device('cuda:0'))
        predict = batch['predict'].to(torch.device('cuda:0'))
        answer = batch['answer'].to(torch.device('cuda:0')) 
        given1 = given.clone().detach().requires_grad_(True)
        predict1 = predict.clone().detach().requires_grad_(True)
        encoder_outs, context = enc(given1)
        guess = dec(encoder_outs, context, predict1)
        mse_loss = nn.MSELoss()
        loss = mse_loss(guess, answer)
    else:
        given1 = batch['given'].to(torch.device('cuda:0'))
        predict1 = batch['predict'].to(torch.device('cuda:0'))
        answer = batch['answer'].to(torch.device('cuda:0')) 
        encoder_outs, context = enc(given1)
        guess = dec(encoder_outs, context, predict1)
        mse_loss = nn.MSELoss()
        loss = mse_loss(guess, answer)
        
    return (loss,given1, predict1 )


In [71]:
from torch import nn 

def GetGradientUsingAutoGrad(enc, dec, TrainingData,batch_size):
    givenbag = []
    predictbag = []
    for batch in DataLoader(TrainingData, batch_size=batch_size, shuffle=True):
        (loss, given1, predict1) = GetLossFunc(batch, enc, dec, autograd=True)
        grad = torch.autograd.grad(outputs=loss, inputs=(given1,predict1), retain_graph=True, allow_unused=True)
        loss.backward()
        givenbag.append(given1.grad)
        predictbag.append(predict1.grad)
    return (grad,enc, dec, givenbag, predictbag)

In [72]:
( grad,enc, dec, givenbag, predictbag) = GetGradientUsingAutoGrad(enc, dec, TrainingData=dataset, batch_size=4096)

In [75]:
predictbag

[tensor([[[ 2.0332e-09,  3.2858e-09, -1.1388e-09,  ..., -4.4638e-09,
           -6.9855e-09, -3.2229e-09],
          [ 2.1751e-09,  7.2334e-09,  1.0810e-09,  ..., -5.4148e-09,
           -8.8567e-09, -5.2784e-09],
          [ 1.4096e-09,  1.0710e-08,  5.8685e-09,  ..., -5.4800e-09,
           -1.1504e-08, -8.4519e-09],
          ...,
          [-2.0426e-08, -7.3111e-08,  1.2361e-07,  ...,  5.2124e-08,
            4.8282e-08,  6.9255e-08],
          [-4.0366e-08, -1.6153e-07,  2.2103e-07,  ...,  8.5868e-08,
            1.6357e-07,  2.2350e-07],
          [-2.1718e-07, -2.5224e-07,  3.6683e-07,  ..., -2.8988e-08,
            3.6659e-07,  5.9498e-07]],
 
         [[ 2.5458e-09,  2.2677e-09, -1.1733e-09,  ..., -3.9874e-09,
           -6.7951e-09, -2.7916e-09],
          [ 3.2655e-09,  4.1472e-09,  1.4556e-10,  ..., -3.9036e-09,
           -8.9973e-09, -3.7405e-09],
          [ 3.2524e-09,  4.8330e-09,  3.7693e-09,  ..., -2.5923e-09,
           -1.2111e-08, -5.5215e-09],
          ...,
    

In [ ]:
onebasket

In [77]:
testbasket = []
for i in range(len(givenzeros)):
    testbasket.append((givenzeros[i]-onebasket[i])/4)
    print(testbasket[i].shape)

NameError: name 'givenzeros' is not defined

In [ ]:
testbasket

In [ ]:
if torch.abs(testbasket[0]) > 100:
    print('yes')
else:
    print('no')

In [78]:
finaldatagrad = []
for i in range(len(testbasket)):
    if torch.max(torch.abs(testbasket[i])) > 0.2:
        testbasket[i] = testbasket[i]/torch.max(torch.abs(testbasket[i]))*0.2

In [79]:
len(testbasket)

0

In [80]:
testbasket

[]

In [81]:
for i in testbasket:
    if torch.max(torch.abs(i)) > 0.2:
        i = i/torch.max(torch.abs(i))*0.2
        print(i.shape)


i

NameError: name 'i' is not defined

In [ ]:
dxp

In [ ]:
len(testbasket)

In [ ]:
givenzeros[0].shape

In [12]:
def GetLossFunc(batch, enc, dec):
        given = batch['given'].to(torch.device('cuda:0'))
        predict = batch['predict'].to(torch.device('cuda:0'))
        answer = batch['answer'].to(torch.device('cuda:0')) 
        encoder_outs, context = enc(given)
        guess = dec(encoder_outs, context, predict)
        mse_loss = nn.MSELoss()
        loss = mse_loss(guess, answer)

        return (loss, given, predict)

In [13]:
import copy
def ObtainListOfParameters(Model):
        l=list()
        for w in Model.parameters():
            l.append(copy.deepcopy(w))
        return l
    
def UpdateModel(Model, Alpha,MaxGrad=1,ListOfParam=None):
    if ListOfParam is None:
        for w in Model.parameters():
            w.data=w.data + Alpha* w.grad
    else:
        i=0
        for w in Model.parameters():
            w.data=w.data +Alpha * ListOfParam[i].grad
            i=i+1

In [ ]:
l=list()
for w in enc.parameters():
    #print(w.grad)
    l.append(w)
    #l.append(copy.deepcopy(w))

In [ ]:
print(l[1].grad)

In [ ]:
enc

In [ ]:
w = ObtainListOfParameters(encoder)
print(w[0].shape)

In [ ]:
w1 = ObtainListOfParameters(encoder)
print(w1[1].grad)

In [ ]:
len(w1)

In [ ]:
for j in range(len)

In [ ]:
import torch
a = torch.rand(10, requires_grad=True)
b = torch.rand(10, requires_grad=True)

output = (2 * a).sum()

gradient = torch.autograd.grad(output, a, retain_graph=True)
print(gradient)
output.backward()


In [ ]:
grad = a.grad
print(grad)

In [ ]:
import torch
net = torch.nn.Linear(2, 2)
d = net.state_dict()
d.update({"custom_stuff": 123})
torch.save(d, "net.pth")

# Load it from the serialized file

new_net = torch.nn.Linear(2, 2)
# By default, strict=True, then you need to force it to False to not raise an error
state_dict = torch.load("net.pth")
custom_stuff = state_dict["custom_stuff"] # Retrieving your stuff after loading
new_net.load_state_dict(state_dict, strict=False) 
# NOTE: you can just del state_dict["custom_stuff"] if you want, then strict can be set True

# Check that the parameters have been loaded as expected

print(net.weight.data.equal(new_net.weight.data))
print(net.bias.data.equal(new_net.bias.data))

In [ ]:
tensor = [grad, gradient]

In [ ]:
tensor

In [ ]:
len(tensor)


In [111]:
def attack_feature(sensorID,num):
    l=list()
    for i in range(0,51*num,51):
        if sensorID==0:
            for j in range(0,5):
                l.append(i+j)
        if sensorID==1:
            for j in range(5,16):
                l.append(i+j)
        if sensorID==2:
            for j in range(16,25):
                l.append(i+j)
        if sensorID==3:
            for j in range(25,34):
                l.append(i+j)
        if sensorID==4:
            for j in range(34,47):
                l.append(i+j)
        if sensorID==5:
            for j in range(47,51):
                l.append(i+j)
    return l

In [114]:
li_attack=attack_feature(3,4)
li_attack

[25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186]